In [7]:
import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [8]:
# Initialize and constants

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
MODEL = 'gpt-4o-mini'
openai = OpenAI()

API key looks good so far


In [85]:
# A class to represent a Webpage

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:
    """
    A utility class to represent a Website that we have scraped, now with links
    """

    def __init__(self, url):
        self.url = url
        response = requests.get(url, headers=headers)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

##### First step: Have GPT-4o-mini figure out which links are relevant
##### Use a call to gpt-4o-mini to read the links on a webpage, and respond in structured JSON.
It should decide which links are relevant, and replace relative links such as "/about" with "https://company.com/about".
We will use "one shot prompting" in which we provide an example of how it should respond in the prompt.

In [51]:
link_system_prompt = "You are provided with a list of links found on a webpage. \
You are able to decide which of the links would be most relevant to include in a brochure about the company, \
such as links to an About page, or a Company page, or Careers/Jobs pages.\n" \
"Do NOT make up any links that are not available on the website.\n"
link_system_prompt += "You should respond in JSON as in this example:"
link_system_prompt += """
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page", "url": "https://another.full.url/careers"}
    ]
}
"""

In [ ]:
print(link_system_prompt)

In [86]:
async def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. \
Do not include Terms of Service, Privacy, email links.\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

In [90]:
def get_links(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)}
      ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    return json.loads(result)

In [ ]:
huggingface = Website("https://huggingface.co")
huggingface.links

##### Second step: Make the brochure

Assemble all the details into another gpt4-o prompt

In [81]:
def get_all_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    links = get_links(url)
    print("Found links:", links)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()
    return result

In [65]:
system_prompt_english = "You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a short brochure about the company for prospective customers, investors and recruits.\
Include details of any relevant information you have about the company. DO NOT make up any information that is not present in the website." \
"Respond in Markdown"

system_prompt_bengali = """
"You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a short brochure about the company for prospective customers, investors and recruits.\
Include details of any relevant information you have about the company. DO NOT make up any information that is not present in the website. \
Respond in Bengali in Markdown. \
Keep any URLs in the brochure in their original text."
"""



In [59]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:1_000] # Truncate if more than 1,000 characters
    return user_prompt

In [61]:
def create_brochure(company_name, url, language):
    if language == "English":
        message = [
            {"role": "system", "content": system_prompt_english},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
        ]
    elif language == "Bengali":
        message = [
            {"role": "system", "content": system_prompt_bengali},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
        ]
    stream = openai.chat.completions.create(
        model=MODEL,
        messages=message,
        stream=True
    )
    result = "" 
    for chunk in stream:
        result += chunk.choices[0].delta.content or ""
        yield result

### Gradio UI

In [62]:
import gradio as gr

In [44]:
force_dark_mode = """
function refresh() {
    const url = new URL(window.location);
    if (url.searchParams.get('__theme') !== 'dark') {
        url.searchParams.set('__theme', 'dark');
        window.location.href = url.href;
    }
}
"""

In [ ]:
gr.Interface(
    fn=create_brochure, 
    inputs= [
                gr.Textbox(label="Company Name"), 
                gr.Textbox(label="Company URL"),
                gr.Dropdown(label="Output Language", choices=["English", "Bengali"])
             ], 
    outputs=[gr.Markdown(label="Brochure")],
    flagging_mode="never", 
    js = force_dark_mode,
    title="AI Brochure Generator",
).launch(share=False)

* Running on local URL:  http://127.0.0.1:7881
* To create a public link, set `share=True` in `launch()`.


Traceback (most recent call last):
  File "/home/python/llm_projects/.venv/lib/python3.12/site-packages/gradio/queueing.py", line 667, in process_events
    response = await route_utils.call_process_api(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/python/llm_projects/.venv/lib/python3.12/site-packages/gradio/route_utils.py", line 349, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/python/llm_projects/.venv/lib/python3.12/site-packages/gradio/blocks.py", line 2274, in process_api
    result = await self.call_function(
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/python/llm_projects/.venv/lib/python3.12/site-packages/gradio/blocks.py", line 1793, in call_function
    prediction = await utils.async_iteration(iterator)
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/python/llm_projects/.venv/lib/python3.12/site-packages/gradio/utils.py", line 760, in async_